In [8]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

import cv2
import skimage

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [9]:
dataDir='/Users/grantwilson/datasets/coco/'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

In [10]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.43s)
creating index...
index created!


In [11]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
accessory outdoor indoor animal sports vehicle food electronic kitchen person appliance furniture


In [12]:
# get all images containing given categories, select one at random
ids = set()
cat_ids = []
for category in ['person','dog','cat', "backpack", "potted plant"]:
    catIds = coco.getCatIds(catNms=category)
    cat_ids.append(catIds[0])
    imgIds = coco.getImgIds(catIds=catIds )
    [ids.add(i) for i in coco.getImgIds(imgIds = imgIds)]
    print(f"{category} {len(imgIds)}")


person 2693
dog 177
cat 184
backpack 228
potted plant 172


In [13]:
ids = list(ids)

In [14]:
images = []
for i in range(len(ids)):
    img = coco.loadImgs(ids[i])[0]
    I = io.imread(img['coco_url'], as_gray=True)
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
    anns = coco.loadAnns(annIds)
    
    for j in range(len(anns)):
        if not anns[j]["iscrowd"]:
            a, b, c, d = [int(k) for k in anns[j]["bbox"]]
            new_im = I[b:b+d, a:a+c]

    
            resized_im = cv2.resize(new_im, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
            im_cat = (anns[j]["category_id"], resized_im)
            images.append(im_cat)
    
    if i % 20 == 1:
        print(f"Iteration: {i}")


Iteration: 1
Iteration: 21
Iteration: 41
Iteration: 61
Iteration: 81
Iteration: 101
Iteration: 121
Iteration: 141
Iteration: 161
Iteration: 181
Iteration: 201
Iteration: 221
Iteration: 241
Iteration: 261
Iteration: 281
Iteration: 301
Iteration: 321
Iteration: 341
Iteration: 361
Iteration: 381
Iteration: 401
Iteration: 421
Iteration: 441
Iteration: 461
Iteration: 481
Iteration: 501
Iteration: 521
Iteration: 541
Iteration: 561
Iteration: 581
Iteration: 601
Iteration: 621
Iteration: 641
Iteration: 661
Iteration: 681
Iteration: 701
Iteration: 721
Iteration: 741
Iteration: 761
Iteration: 781
Iteration: 801
Iteration: 821
Iteration: 841
Iteration: 861
Iteration: 881
Iteration: 901
Iteration: 921
Iteration: 941
Iteration: 961
Iteration: 981
Iteration: 1001
Iteration: 1021
Iteration: 1041
Iteration: 1061
Iteration: 1081
Iteration: 1101
Iteration: 1121
Iteration: 1141
Iteration: 1161
Iteration: 1181
Iteration: 1201
Iteration: 1221
Iteration: 1241
Iteration: 1261
Iteration: 1281
Iteration: 1301


In [43]:
image_category = np.array([i[0] for i in images])
image = np.array([i[1] for i in images])

In [45]:
im_hog = []
for i in range(0, image.shape[0]):
    im_hog.append(skimage.feature.hog(image[i], visualize=True)[1])

In [46]:
np.save("hog.npy", im_hog)
np.save("image.npy", image)
np.save("image_category.npy", image_category)

In [49]:
np.savez("data", image_category, image, im_hog)

In [48]:
np.load("image.npy").shape

(11910, 128, 128)

In [ ]:
df = pd.DataFrame(images, columns=["Category", "image"])

In [ ]:
df.to_csv("preprocessed_images.csv", index=False)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
img = coco.loadImgs(ids[np.random.randint(0,len(ids))])[0]

# load and display image
# I = io.imread('%s/images/%s/%s'%(dataDir,dataType,img['file_name']))
# use url to load image
I = io.imread(img['coco_url'], as_gray=True)
plt.axis('off')
plt.imshow(I, cmap="gray")
plt.show()

In [ ]:
# load and display instance annotations
plt.imshow(I, cmap="gray"); plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)

In [ ]:
anns[0].keys()

In [ ]:
anns[6]["iscrowd"]

In [ ]:
a, b, c, d = [int(i) for i in anns[0]["bbox"]]

In [ ]:
print((a, b, c, d))

In [ ]:
im_collection = []
for i in range(len(anns)):
    a, b, c, d = [int(j) for j in anns[i]["bbox"]]
    im_collection.append(I[b:b+d, a:a+c])

In [ ]:
# create multi-row figure
plt.figure(figsize=(25, 3))


# add images to the figure from the stack
for i in range(1, len(im_collection)+1):
    plt.subplot(1, len(im_collection), i)

    # don't forget to set the cmap, vmin, vmax, and axis off
    plt.imshow(im_collection[i-1], cmap="gray")
    plt.axis("off")


plt.show()

In [ ]:
# create multi-row figure
plt.figure(figsize=(25, 3))

resized_images = []
# add images to the figure from the stack
for i in range(1, len(im_collection)+1):
    plt.subplot(1, len(im_collection), i)

    # don't forget to set the cmap, vmin, vmax, and axis off
    resized_im = cv2.resize(im_collection[i-1], dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    resized_images.append(resized_im)
    plt.imshow(resized_im, cmap="gray")
    plt.axis("off")
    
    plt.title(f"Person {i-1}")


plt.show()



In [ ]:
PERSON_SELECTOR = 0

In [ ]:
a = np.float32(resized_images[PERSON_SELECTOR])

gx = cv2.Sobel(a, cv2.CV_32F, 1, 0, ksize=1)
gy = cv2.Sobel(a, cv2.CV_32F, 0, 1, ksize=1)

In [ ]:
mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
angle = angle%180

In [ ]:
plt.imshow(angle, cmap="gray")

In [ ]:
plt.imshow(mag, cmap="gray")

In [ ]:
# initialize COCO api for person keypoints annotations
annFile = '{}/annotations/person_keypoints_{}.json'.format(dataDir,dataType)
coco_kps=COCO(annFile)

In [ ]:
# load and display keypoints annotations
plt.imshow(I, cmap="gray"); plt.axis('off')
ax = plt.gca()
annIds = coco_kps.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
anns = coco_kps.loadAnns(annIds)
coco_kps.showAnns(anns)

In [ ]:
# initialize COCO api for caption annotations
annFile = '{}/annotations/captions_{}.json'.format(dataDir,dataType)
coco_caps=COCO(annFile)

In [ ]:
# load and display caption annotations
annIds = coco_caps.getAnnIds(imgIds=img['id']);
anns = coco_caps.loadAnns(annIds)
coco_caps.showAnns(anns)
plt.imshow(I, cmap="gray"); plt.axis('off'); plt.show()